In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image, ImageOps
import cv2
import time
import os

In [ ]:
mtcnn0 = MTCNN(image_size=160, margin=0, keep_all=False, min_face_size=40)
mtcnn = MTCNN(image_size=160, margin=0, keep_all=True, min_face_size=40)
resnet = InceptionResnetV1(pretrained='vggface2').cuda().eval()
resnet.classify = False


if not os.path.exists('./data_updated.pt'):
    dataset = datasets.ImageFolder('datasets/LFW') # photos folder path
    idx_to_class = {i:c for c,i in dataset.class_to_idx.items()}

    def collate_fn(x):
      return x[0]

    loader = DataLoader(dataset, collate_fn=collate_fn)

    name_list = [] 
    embedding_list = [] 

    for img, idx in loader:
        face, prob = mtcnn0(img, return_prob=True)
        if face is not None and prob > 0.90:
            emb = resnet(face.unsqueeze(0).cuda())
            embedding_list.append(emb.detach())
            name_list.append(idx_to_class[idx])

    # save data
    data = [embedding_list, name_list]
    torch.save(data, './data_updated.pt') # saving data.pt file